In [1]:
#Open API Fine Tuning Dashboard URL: https://platform.openai.com/finetune/

In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)


In [3]:
with open("data/training_data.jsonl", "rb") as f:
    training_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded training file ID:", training_file_obj.id)

with open("data/validation_data.jsonl", "rb") as f:
    validation_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded validation file ID:", validation_file_obj.id)

Uploaded training file ID: file-PeL3r5oXdFsvS3hjTbj7gQ
Uploaded validation file ID: file-P2tH76PAJw2eniwGjJBTNS


In [4]:
# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file_obj.id,       # The file ID you uploaded
    validation_file=validation_file_obj.id,     # Optional
    model="gpt-4.1-nano-2025-04-14",               # or "gpt-3.5-turbo", etc.
    suffix="brand-customer-support"       # Optional model name suffix
)

print("Fine-tune job created:", job.id)

Fine-tune job created: ftjob-kgtSzWeDFomsD3sVId0jzat3


In [5]:
jobs = client.fine_tuning.jobs.list(limit=5)
for j in jobs.data:
    print(j.id, j.status, j.fine_tuned_model)

ftjob-kgtSzWeDFomsD3sVId0jzat3 validating_files None
ftjob-7k4CgFexoyMbFTFot58dxbsc succeeded ft:gpt-4.1-nano-2025-04-14:personal:brand-customer-support:CjrA3nIN


In [6]:
def ask_question_without_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

def ask_question_with_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="ft:gpt-4.1-nano-2025-04-14:personal:brand-customer-support:CjrA3nIN",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

In [8]:
import time
while True:
    # Ask user for a question
    user_prompt = input("Ask something: ")

    if (user_prompt.lower() != 'quit'):
        # Get and print the response
        response = ask_question_without_finetuning(user_prompt)
        print("\n[WITHOUT FINE TUNING] OpenAI says:", response)

        response = ask_question_with_finetuning(user_prompt)
        print("\n[WITH FINE TUNING] OpenAI says:", response)

        # add delay of 3 seconds
        time.sleep(3)
    else:
        break    

User asked: My login is not working
ChatCompletion(id='chatcmpl-CjrcDtuuu5cBBPRxviLEkaItQkVOX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sorry you’re having trouble. Here’s a quick triage you can try:\n\n- Check your credentials: correct username/email and password? Is Caps Lock on?\n- Try Forgot password to reset. Check your email (and spam) for the reset link.\n- If you use 2FA, ensure you’re entering the code from your authenticator/app or SMS, and have backup codes if needed.\n- Clear browser issues: try incognito/private mode, clear cache/cookies, or try a different browser/device.\n- Disable VPN or firewall that might block login.\n- If there’s an outage, check the service’s status page or social accounts.\n\nIf none of that helps, please tell me:\n- Which service or app are you trying to log into?\n- Any exact error message you see?\n- Are you using 2FA?\n- Are you on a computer or phone? Which browser/app?\n\nWith thos